In [13]:
import json
import os
from os.path import join
from glob import glob
import markdown
import base64

from tensorboard.backend.event_processing.tag_types import IMAGES

from pcgrllm.utils.storage import INFERENCE_DIR


In [14]:
SOURCE_DIR = 'src'
REWARD_FUNCTION_DIR = 'reward_functions'
FEEDBACK_DIR = 'feedback'
INFERENCE_DIR = 'inference'
IMAGES_DIR = 'images'
NUMPY_DIR = 'numpy'
REWARD_FUNCTION_SUBDIR_FORMAT = 'reward_outer_{iteration}_inner_1'
TRIAL_FILENAME_FORMAT = 'reward_outer_{iteration}_inner_1_trial_{trial}.json'
FEEDBACK_FILENAME_FORMAT = 'feedback_log_iter_{iteration}.json'
ITERATION_DIR_FORMAT = 'iteration_{iteration}'

In [15]:
exp_lists = glob(join(SOURCE_DIR, '*'))
exp_lists

['src/pe-cotsc_it-5_fit-vit_exp-def_chr-N_s-0',
 'src/pe-cot_it-5_fit-vit_exp-def_chr-C_s-1',
 'src/pe-cot_it-5_fit-vit_exp-def_chr-F_s-1',
 'src/pe-cot_it-5_fit-vit_exp-def_chr-D_s-1',
 'src/pe-io_it-1_fit-vit_exp-def_chr-F_s-0',
 'src/pe-tot_it-6_fit-vit_exp-def_chr-N_s-0',
 'src/pe-tot_it-6_fit-vit_exp-def_chr-M_s-0',
 'src/pe-io_it-1_fit-vit_exp-def_chr-D_s-0',
 'src/pe-cot_it-5_fit-vit_exp-def_chr-N_s-2',
 'src/pe-io_it-1_fit-vit_exp-def_chr-A_s-0',
 'src/pe-cot_it-5_fit-vit_exp-def_chr-M_s-2',
 'src/pe-io_it-1_fit-vit_exp-def_chr-C_s-0',
 'src/pe-cot_it-5_fit-vit_exp-def_chr-MakeXlevel95_s-4',
 'src/pe-tot_it-6_fit-vit_exp-def_chr-F_s-0',
 'src/pe-io_it-1_fit-vit_exp-def_chr-N_s-0',
 'src/pe-tot_it-6_fit-vit_exp-def_chr-D_s-0',
 'src/pe-io_it-1_fit-vit_exp-def_chr-M_s-0',
 'src/pe-cot_it-5_fit-vit_exp-def_chr-C_s-2',
 'src/pe-cot_it-5_fit-vit_exp-def_chr-F_s-2',
 'src/pe-cot_it-5_fit-vit_exp-def_chr-D_s-2',
 'src/pe-tot_it-6_fit-vit_exp-def_chr-C_s-0',
 'src/pe-cot_it-5_fit-vit_e

In [16]:
def _get_trial_number(filename):
    import re
    match = re.search(r'trial_(\d+)\.json', filename)

    if match:
        trial_number = int(match.group(1))
        return trial_number
    else:
        raise ValueError(f'Could not extract trial number from {filename}')

In [17]:
import os
import json
import base64
import re
from glob import glob
from os.path import join
import markdown

def _get_trial_number(filename):
    """Extracts trial number using regex."""
    import re
    match = re.search(r'trial_(\d+)\.json', filename)
    
    if match is None:
        raise ValueError(f'Could not extract trial number from {filename}')
    return int(match.group(1)) if match else None

def generate_html_style():
    """Returns CSS styles for the HTML."""
    return '''
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap');
        body {
            font-family: 'Roboto', sans-serif;
            background-color: #f7f8fa;
            margin: 0;
            padding: 20px;
            color: #2c3e50;
        }
        .container {
            margin-bottom: 30px;
            padding: 20px;
            background-color: #ffffff;
            border-radius: 12px;
            box-shadow: 0 4px 12px rgba(0, 0, 0, 0.1);
        }
        h1, h2 {
            color: #34495e;
            margin-bottom: 15px;
        }
        h3 {
            color: #2c3e50;
            margin-bottom: 10px;
        }
        .message-box {
            padding: 15px;
            border-radius: 10px;
            margin: 10px 0;
            box-shadow: 0 2px 8px rgba(0, 0, 0, 0.1);
            white-space: pre-wrap;
            word-wrap: break-word;
        }
        .message-container {
            margin-bottom: 20px;
        }
        .message.user .message-box {
            background-color: #dce8f6;  /* Soft Blue */
            color: #1c2833;
            align-self: flex-start;
        }
        .message.assistant .message-box {
            background-color: #f4e1d2;  /* Soft Peach */
            color: #1c2833;
            align-self: flex-end;
        }
        img {
            max-width: 100%;
            height: auto;
            margin-top: 10px;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
        }
    </style>
    '''

for exp_list in exp_lists:
    exp_name = exp_list.split('/')[-1]
    title = f"Experiment Results - {exp_name}"
    outputs = [f'<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>{title}</title>{generate_html_style()}</head><body>']
    outputs.append(f'<h1>{exp_list}</h1>')

    exps = glob(join(exp_list, 'iteration_*'))
    exps = sorted(exps, key=lambda x: int(x.split('_')[-1]))

    iteration_nums = sorted([int(exp.split('_')[-1]) for exp in exps])

    for iter_num in iteration_nums:
        outputs.append(f'<div class="container"><h2>Iteration {iter_num}</h2>')

        reward_dir = join(exp_list, REWARD_FUNCTION_DIR, REWARD_FUNCTION_SUBDIR_FORMAT.format(iteration=iter_num))
        reward_trials = glob(join(reward_dir, 'reward_*.json'))
        reward_trials = [f for f in reward_trials if not re.search(r'branch', f)]


        trial_nums = sorted(list(map(_get_trial_number, reward_trials)))
        
        for trial_num in trial_nums:
            trial_json_path = join(reward_dir, TRIAL_FILENAME_FORMAT.format(iteration=iter_num, trial=trial_num))
            trial_file = json.load(open(trial_json_path))

            outputs.append(f'<div class="message-container"><h3>Trial {trial_num}</h3>')
            for req_content in trial_file['request']:
                markdown_content = markdown.markdown(req_content["content"], extensions=['extra'])
                outputs.append(f'''
                <div class="message user">
                    <div class="message-box">{markdown_content}</div>
                </div>
                ''')

            res_content = markdown.markdown(trial_file['response'], extensions=['extra'])
            outputs.append(f'''
            <div class="message assistant">
                <div class="message-box">{res_content}</div>
            </div>
            </div>  <!-- Close message-container -->
            ''')

        outputs.append('<div class="message-container"><h3>Result numpy/image files</h3>')

        images = glob(join(exp_list, ITERATION_DIR_FORMAT.format(iteration=iter_num), INFERENCE_DIR, IMAGES_DIR, '*.png'))
        for image in images:
            with open(image, 'rb') as img_file:
                encoded_string = base64.b64encode(img_file.read()).decode('utf-8')
            img_tag = f'<img src="data:image/png;base64,{encoded_string}" alt="{os.path.basename(image)}" />'
            outputs.append(img_tag)

        feedback_dir = join(exp_list, ITERATION_DIR_FORMAT.format(iteration=iter_num), FEEDBACK_DIR)
        feedback_json = glob(join(feedback_dir, FEEDBACK_FILENAME_FORMAT.format(iteration=iter_num)))

        if feedback_json:
            feedback_file = json.load(open(feedback_json[0]))['chats'][0]
            user_message = markdown.markdown(feedback_file["user_message"], extensions=['extra'])
            assistant_message = markdown.markdown(feedback_file["assistant_message"], extensions=['extra'])

            outputs.append(f'''
            <div class="message user">
                <div class="message-box">{user_message}</div>
            </div>
            <div class="message assistant">
                <div class="message-box">{assistant_message}</div>
            </div>
            </div>  <!-- Close message-container -->
            ''')

        outputs.append('</div>  <!-- Close container -->')

    outputs.append('</body></html>')  # Close HTML document

    output = '\n'.join(outputs)

    os.makedirs('dest', exist_ok=True)
    with open(f'dest/{exp_name}.html', 'w') as f:
        f.write(output)


In [18]:
reward_trials = [f for f in reward_trials if not re.search(r'branch', f)]
